In [21]:
import numpy as np
import math
import tensorflow as tf


In [2]:
def uSoln(u0, ds, inFilt, pFilt, b):
    inChan, row, col = u0.shape
    tMax, outChan = inFilt.shape[0], inFilt.shape[2]
    u = np.zeros((tMax, outChan, row, col))
    x = np.zeros((tMax, outChan, row, col))
    # y=np.zeros((tMax,outChan,row,col))
    bzero = np.zeros((outChan))
    # print('outChan=',outChan,', u shape:',u.shape)

    # print('inFilt shape=',inFilt.shape,', u0 shape:',u0.shape)
    u[0] = conv(u0, inFilt[0], bzero)
    # print('u0',u[0])
    for h in range(tMax - 1):
        # print('h=',h)
        x[h] = act(u[h])
        zInt = np.array([conv(x[s], pFilt[s, h], b[h] / ((h + 1) * ds)) for s in range(h + 1)])
        z = trapInt(zInt, ds)
        u[h + 1] = z + conv(u0, inFilt[h + 1], bzero)
        # print('actU=',actU)
        # print('y[',h,']=',y[h])
    #        print('zInt=',zInt)
    # print('z=',z)
    # print('u[',h+1,']=',u[h+1])
    x[tMax - 1] = act(u[tMax - 1])
    return x, u

In [3]:
def im2col(image, kerRow, kerCol, st):
    """
    Returns:
      col: (new_h*new_w,kerRow*KerCol*imageChan) matrix,
            each column is a cube that will convolve with a filter
            new_h=(imageRow-kerRow)//st+1, new_w=(imagrCol-kerCol)//st+1
    """

    chan, row, col = image.shape
    new_h = (row - kerRow) // st + 1
    new_w = (col - kerCol) // st + 1
    col = np.zeros([new_h * new_w, chan * kerRow * kerCol])

    for i in range(new_h):
        for j in range(new_w):
            patch = image[..., i * st:i * st + kerRow, j * st:j * st + kerCol]
            col[i * new_w + j, :] = np.reshape(patch, -1)
    return col

In [4]:
def mul(num):
    p = 1
    for m in num:
        p *= m
    # print('p=',p)
    return p

In [5]:
def col2im(ma, h_prime, w_prime, C):
    """
      Args:
      ma: (h_prime*w_prime*w,F) matrix, each col should be reshaped to C*h_prime*w_prime when C>0, or h_prime*w_prime when C = 0
      h_prime: reshaped filter height
      w_prime: reshaped filter width
      C: reshaped filter channel, if 0, reshape the filter to 2D, Otherwise reshape it to 3D
    Returns:
      if C == 0: (F,h_prime,w_prime) matrix
      Otherwise: (F,C,h_prime,w_prime) matrix
    """
    F = ma.shape[1]
    if (C == 1):
        out = np.zeros([F, h_prime, w_prime])
        for i in range(F):
            col = ma[:, i]
            out[i, :, :] = np.reshape(col, (h_prime, w_prime))
    else:
        out = np.zeros([F, C, h_prime, w_prime])
        for i in range(F):
            col = ma[:, i]
            out[i, :, :] = np.reshape(col, (C, h_prime, w_prime))

    return out

In [6]:
def conv(image, filt, b, st=1):
    imChan, imRow, imCol = image.shape
    inChan, outChan, kerRow, kerCol = filt.shape
    r = (imRow - 1) // st + 1
    c = (imCol - 1) // st + 1
    out = np.zeros((outChan, r, c))

    imPad = np.pad(image, ((0, 0), ((kerRow - 1) // 2, (kerRow - 1) // 2),
                           ((kerCol - 1) // 2, (kerCol - 1) // 2)))
    imMat = im2col(imPad, kerRow, kerCol, st)
    filtCol = np.reshape(filt, (outChan, -1))
    outCol = imMat.dot(filtCol.T) + b
    out = col2im(outCol, r, c, 1)
    return out

In [7]:
def act(layer):
    chan, row, col = layer.shape
    out = np.zeros((chan, row, col))

    for c in range(chan):
        for i in range(row):
            for j in range(col):
                if layer[c, i, j] >= 0:
                    out[c, i, j] = actWt[1] * layer[c, i, j]
                else:
                    out[c, i, j] = actWt[0] * layer[c, i, j]
    return out

In [8]:
def actInv(layer):
    chan, row, col = layer.shape
    out = np.zeros((chan, row, col))

    for c in range(chan):
        for i in range(row):
            for j in range(col):
                if layer[c, i, j] >= 0:
                    out[c, i, j] = layer[c, i, j] / actWt[1]
                else:
                    out[c, i, j] = layer[c, i, j] / actWt[0]
    return out

In [9]:
def actD(layer):
    chan, row, col = layer.shape
    out = np.zeros((chan, row, col))

    for c in range(chan):
        for i in range(row):
            for j in range(col):
                if layer[c, i, j] >= 0:
                    out[c, i, j] = actWt[1]
                else:
                    out[c, i, j] = actWt[0]
    return out

In [10]:
def trapInt(vals, ds):
    return ds / 2 * (np.sum(vals, axis=0) + np.sum(vals[1:-1], axis=0))
    #trapezoidal rule: (1/2)*ds*(f(x_0) + 2f(x_1) + 2f(x_2) + ... + 2f(x_n-1) + f(x_n))

In [12]:
def trapezoidIntegral(vals, ds):
    pass1 = tf.reduce_sum(vals, axis=0)
    pass2 = tf.reduce_sum(vals[1:-1], axis=0)
    return ds/2.0*(pass1+pass2)
def TFconv(image, filt, b):
    return tf.nn.convolution(input=image, filters=filt, padding="SAME") + b

In [40]:
def TFuSoln(raw_u0, ds, inFilt, pFilt, b):
    bzero = tf.zeros([inFilt.shape[4]])
    u0 = TFconv(raw_u0, inFilt[0], bzero)
    #for loop begins, h=0
    x0 = tf.nn.leaky_relu(u0, alpha=0.1)
    zInt0 = TFconv(x0, pFilt[0,0], b[0] / ((0+1)*ds))
    z0 = trapezoidIntegral(zInt0, ds)
    u1 = z0 + TFconv(raw_u0, inFilt[1], bzero)
    #for loop begins, h=1
    x1 = tf.nn.leaky_relu(u1, alpha=0.1)
    zInt1_0 = TFconv(x0, pFilt[0,1], b[1] / ((1+1)*ds)) #s=0
    zInt1_1 = TFconv(x1, pFilt[1,1], b[1] / ((1+1)*ds)) #s=1
    zInt1 = zInt1_0 + zInt1_1
    z1 = trapezoidIntegral(zInt1, ds)
    u2 = z1 + TFconv(raw_u0, inFilt[2], bzero)
    #for loop begins, h=2
    x2 = tf.nn.leaky_relu(u2, alpha=0.1)
    zInt2_0 = TFconv(x0, pFilt[0,2], b[2] / ((2+1)*ds)) #s=0
    zInt2_1 = TFconv(x1, pFilt[1,2], b[2] / ((2+1)*ds)) #s=1
    zInt2_2 = TFconv(x2, pFilt[2,2], b[2] / ((2+1)*ds)) #s=2
    zInt2 = zInt2_0 + zInt2_1 + zInt2_2
    z2 = trapezoidIntegral(zInt2, ds)
    u3 = z2 + TFconv(raw_u0, inFilt[3], bzero)
    #end loop
    x3 = tf.nn.leaky_relu(u3, alpha=0.1)
    
    x = tf.stack([x0,x1,x2,x3])
    u = tf.stack([u0,u1,u2,u3])
    
    return x,u


In [123]:
t0 = 3
t1 = t0+1
dt = 1.0/t0
input_shape = (1,5,5)
filter_shape = (1,1,3,3)



#np.random.seed(42)
actWt = [.1, 1] #leakyRelU: first number is slope left of origin, second number is slope right of origin

rng = np.random.default_rng(42)
uIn = rng.random(input_shape, dtype=np.float32)
iFilt = rng.random((t1,*filter_shape), dtype=np.float32)
paramFilt = rng.random((t1,t1,*filter_shape), dtype=np.float32)
biasVec = rng.random((t1,1), dtype=np.float32)

xOut, uOut = uSoln(uIn, dt, iFilt, paramFilt, biasVec)
print(xOut)

[[[[ 0.51498272  1.27649589  1.1055959   0.9759438   0.48584086]
   [ 1.68169929  2.34558572  2.28128301  2.10167874  0.84754259]
   [ 1.44620363  2.50932339  2.29710814  2.02582622  1.13322457]
   [ 1.84604615  2.30006102  2.88054014  2.57576111  1.71913215]
   [ 1.12747911  1.33761076  1.77357502  1.65354777  1.57396889]]]


 [[[ 1.78319511  2.11366493  2.54893388  1.83954434  0.95444824]
   [ 3.07479777  4.54830764  4.20990784  3.93714221  1.89299125]
   [ 3.12063286  4.94908808  4.44072544  4.42363938  2.3203553 ]
   [ 3.24411366  4.27407309  5.25468194  4.79570856  3.13236746]
   [ 1.74309188  2.85732423  3.57468866  2.82905223  2.5140795 ]]]


 [[[ 3.07593597  4.69323918  4.57032454  3.89175476  2.18694832]
   [ 5.09171822  7.2259      7.67881208  6.26969443  3.80258487]
   [ 5.33198863  8.91217132  8.35631047  7.55551793  5.15421687]
   [ 5.48345957  7.45774054  8.10733055  8.01567449  5.49496459]
   [ 3.25044943  4.46620158  5.68629134  5.64797824  3.85269012]]]


 [[[ 5.408404

In [124]:
#TFuIn = tf.transpose()
TFuIn = tf.expand_dims(tf.transpose(tf.convert_to_tensor(uIn, dtype=tf.float32), perm=[1,2,0]), 0)
TFiFilt = tf.transpose(tf.convert_to_tensor(iFilt, dtype=tf.float32), perm=[0,3,4,1,2])
TFpFilt = tf.transpose(tf.convert_to_tensor(paramFilt, dtype=tf.float32), perm=[0,1,4,5,2,3])
TFbiasVec = tf.convert_to_tensor(biasVec, dtype=tf.float32)

TFxOut, TFuOut = TFuSoln(TFuIn, dt, TFiFilt, TFpFilt, TFbiasVec)
print(tf.transpose(TFxOut, perm=[4,0,1,2,3]))


tf.Tensor(
[[[[[ 0.5149827   1.2764959   1.1055958   0.9759438   0.48584086]
    [ 1.6816992   2.3455856   2.2812831   2.1016786   0.84754264]
    [ 1.4462036   2.5093234   2.2971082   2.0258262   1.1332246 ]
    [ 1.8460461   2.300061    2.8805401   2.5757613   1.7191321 ]
    [ 1.1274791   1.3376107   1.773575    1.6535478   1.5739689 ]]]


  [[[ 1.7831951   2.1136649   2.548934    1.8395444   0.9544482 ]
    [ 3.0747976   4.548308    4.209908    3.9371424   1.8929913 ]
    [ 3.1206326   4.949088    4.4407253   4.4236393   2.3203554 ]
    [ 3.2441137   4.274073    5.2546816   4.7957087   3.1323674 ]
    [ 1.7430918   2.8573241   3.5746884   2.8290522   2.5140796 ]]]


  [[[ 3.0759358   4.693239    4.570325    3.891755    2.1869485 ]
    [ 5.0917187   7.2258997   7.678812    6.2696943   3.8025851 ]
    [ 5.3319883   8.912171    8.356311    7.555518    5.154217  ]
    [ 5.4834595   7.457741    8.10733     8.015675    5.4949646 ]
    [ 3.2504497   4.466202    5.6862917   5.6479783   3.8